## Dataset Combination
---------------
This notebook provides the framework for combining the common features of multiple datasets 

In [1]:
#imports
from DatasetTools import loadDataset,Datasets,findCommonFeatures,combineDatasets
from Loader.CombinedDataset import CombinationMethod
import pandas as pd

### Global variables

In [2]:
calcFeatures = True
outputPath = "../datasets/Combined/sample.csv"

### Loading datasets
To load a dataset, simply call the *loadDataset* method from DatasetTools and specify which dataset is being loaded along with the path the the file. You can also provide a name for the dataset as a note on the combined.

In [3]:

usb1 = loadDataset(Datasets.USBIDS_2021,'../datasets/USBIDS21/Slowhttptest-NoDefense.csv',"USB-IDS Slowhttptest",calcFeatures=calcFeatures)
usb2 = loadDataset(Datasets.USBIDS_2021,'../datasets/USBIDS21/REGULAR.csv',"USB-IDS Regular",calcFeatures=calcFeatures)

UNSW1 = loadDataset(Datasets.UNSW_NB15_2015,"../datasets/UNSW-NB15/UNSW-NB15_1.csv","UNSW-NB15_1",calcFeatures=calcFeatures)
UNSW2 = loadDataset(Datasets.UNSW_NB15_2015,"../datasets/UNSW-NB15/UNSW-NB15_2.csv","UNSW-NB15_2",calcFeatures=calcFeatures)

loading Datasets.USBIDS_2021 from "../datasets/USBIDS21/Slowhttptest-NoDefense.csv" with dataset name: USB-IDS Slowhttptest
loading Datasets.USBIDS_2021 from "../datasets/USBIDS21/REGULAR.csv" with dataset name: USB-IDS Regular
loading Datasets.UNSW_NB15_2015 from "../datasets/UNSW-NB15/UNSW-NB15_1.csv" with dataset name: UNSW-NB15_1
loading Datasets.UNSW_NB15_2015 from "../datasets/UNSW-NB15/UNSW-NB15_2.csv" with dataset name: UNSW-NB15_2


### Combining datasets
There are two methods of combining datasets, sequentially and interlaced.
- Sequentially: datasets are concatenated together in the order they are passed in.
- Interlaced: datasets are mixed together based on the relative timestamp in each dataset.

In [4]:
comb = combineDatasets(usb1, usb2, UNSW1, UNSW2, method = CombinationMethod.SEQUENTIAL)


In [5]:
combUSB = combineDatasets(usb1, usb2, method = CombinationMethod.SEQUENTIAL)
combUNSW = combineDatasets(UNSW1, UNSW2, method = CombinationMethod.SEQUENTIAL)
comb = combineDatasets(combUNSW, combUSB, method = CombinationMethod.SEQUENTIAL)

In [6]:
temp = next(comb)

AttributeError: 'CombinedDataset' object has no attribute 'name'